### Práctica Modelos AR y MA 

1.- Diseñe una clase para un modelo __AR(p)__ y una para un modelo __MA(q)__. Cada clase debe permitir

1. Recibir el orden para el que sera entrenado. (p o q según sea el caso)
2. Ajustar los parámetros usando los algoritmos de Durvin-Levinson o Innovations según sea el caso. 
3. Determinar el polinomio asociado a cada modelo y sus raices.
4. Determinar si el modelo es invertible o causal según sea el caso.
5. Obtener los valores de PACF cuando aplique.

In [1]:
import numpy as np
from scipy.linalg import toeplitz

class AR:
    def __init__(self, p):
        self.p = p
        self.coef = np.zeros(p)
        self.sigma2 = 0
        
    def fit(self, data):
        T = len(data)
        y = np.zeros(T - self.p)
        X = np.zeros((T - self.p, self.p))
        for t in range(self.p, T):
            y[t - self.p] = data[t]
            X[t - self.p, :] = data[t - self.p:t]
        self.coef = np.linalg.inv(X.T @ X) @ X.T @ y
        eps = y - X @ self.coef
        self.sigma2 = np.sum(eps ** 2) / (T - self.p)
        return self
    
    def ar_poly(self):
        return np.concatenate(([1], -self.coef))
    
    def ar_roots(self):
        return np.roots(self.ar_poly())
    
    def is_invertible(self):
        return np.all(np.abs(self.ar_roots()) > 1)
    
    def pacf(self):
        pacf = np.zeros(self.p + 1)
        pacf[0] = 1
        pacf[1:] = self.coef
        return pacf


class MA:
    def __init__(self, q):
        self.q = q
        self.coef = np.zeros(q)
        self.sigma2 = 0
        
    def fit(self, data):
        T = len(data)
        eps = np.zeros(T - self.q)
        theta = np.zeros(self.q)
        for t in range(self.q, T):
            eps[t - self.q] = data[t] - theta @ data[t - self.q:t][::-1]
            theta -= np.linalg.inv(toeplitz(data[t - self.q:t])) @ eps[t - self.q] * np.array([1] + list(theta))
        self.coef = theta
        self.sigma2 = np.sum(eps ** 2) / (T - self.q)
        return self
    
    def ma_poly(self):
        return np.concatenate(([1], self.coef))
    
    def ma_roots(self):
        return np.roots(self.ma_poly())
    
    def is_causal(self):
        return np.all(np.abs(self.ma_roots()) > 1)
    
    def pacf(self):
        return np.zeros(self.q + 1)


2.- Probar las clases en un dataset (todavia no sé cuál)

In [2]:
import pandas as pd
from statsmodels.graphics.tsaplots import plot_pacf

data = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv', 
                   index_col='Month', parse_dates=['Month'])
train = data.loc[:'1958']
test = data.loc['1959']


In [4]:
ar_model = AR(p=3).fit(train)
print("Coeficientes AR:", ar_model.coef)
print("Polinomio AR:", ar_model.ar_poly())
print("Raíces AR:", ar_model.ar_roots())
print("¿El modelo AR es invertible?:", ar_model.is_invertible())
print("PACF AR:", ar_model.pacf())




KeyError: 3

In [5]:
ma_model = MA(q=3).fit(train)
print("Coeficientes MA:", ma_model.coef)
print("Polinomio MA:", ma_model.ma_poly())
print("Raíces MA:", ma_model.ma_roots())
print("¿El modelo MA es causal?:", ma_model.is_causal())
print("PACF MA:", ma_model.pacf())

KeyError: 3

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 6))

# Serie de tiempo original
ax.plot(test.index, test.values, label='Observaciones')

# Predicciones del modelo AR(3)
ar_pred = ar_model.ar_poly()
ar_pred = np.concatenate((ar_pred, np.zeros(len(test)-len(ar_pred))))
ar_pred = np.convolve(test.values, ar_pred[::-1], mode='valid')
ax.plot(test.index, ar_pred, label='AR(3)')

# Predicciones del modelo MA(3)
ma_pred = ma_model.ma_poly()
ma_pred = np.concatenate(([1], np.zeros(len(test)-1)))
ma_pred = np.convolve(test.values, ma_pred[::-1], mode='valid')
ax.plot(test.index, ma_pred, label='MA(3)')

ax.legend()
plt.show()
